In [1]:
# IPL Ball-by-Ball Analytics with Real Deliveries Dataset (260,921 rows!)
# Modules: Data Formats, Data Modelling, Descriptive Statistics, Queries

import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical operations
import matplotlib.pyplot as plt  # For creating visualizations
import seaborn as sns  # For statistical visualizations
from collections import Counter  # For counting occurrences

print("🏏 IPL Ball-by-Ball Analytics - Real Kaggle Dataset (260,921 deliveries!)")
print("=" * 70)

# ==========================================
# MODULE 1: LOADING & EXPLORING REAL DATA
# ==========================================

print("\n📊 MODULE 1: LOADING THE REAL IPL DELIVERIES DATASET")
print("-" * 50)

# Load the actual deliveries.csv file
# Make sure deliveries.csv is in the same folder as your Python script
try:
    df_deliveries = pd.read_csv('deliveries.csv')
    print("✅ Successfully loaded deliveries.csv!")
    print(f"📈 Dataset size: {df_deliveries.shape[0]:,} deliveries, {df_deliveries.shape[1]} columns")
    
except FileNotFoundError:
    print("❌ Error: deliveries.csv not found!")
    print("Please ensure deliveries.csv is in the same folder as this script")
    print("You can download it from: https://www.kaggle.com/datasets/patrickb1912/ipl-complete-dataset-20082020")
    exit()

# Understanding our massive dataset
print(f"\n🔍 DATASET OVERVIEW:")
print(f"• Total deliveries: {len(df_deliveries):,}")
print(f"• Unique matches: {df_deliveries['match_id'].nunique():,}")
print(f"• Seasons covered: Likely 2008-2020+ (based on common Kaggle datasets)")
print(f"• Teams involved: {df_deliveries['batting_team'].nunique()} teams")

# Column information
print(f"\n📋 COLUMNS IN OUR DATASET:")
print("Column Name          | Data Type    | Description")
print("-" * 65)

column_descriptions = {
    'match_id': 'Unique match identifier',
    'inning': '1st or 2nd innings', 
    'batting_team': 'Team currently batting',
    'bowling_team': 'Team currently bowling',
    'over': 'Over number (0-19)',
    'ball': 'Ball in over (1-6+)',
    'batter': 'Batsman on strike',
    'bowler': 'Current bowler',
    'non_striker': 'Non-striker batsman',
    'batsman_runs': 'Runs by batsman',
    'extra_runs': 'Extra runs (wides, byes, etc.)',
    'total_runs': 'Total runs this ball',
    'extras_type': 'Type of extra',
    'is_wicket': 'Wicket taken (0/1)',
    'player_dismissed': 'Dismissed player name',
    'dismissal_kind': 'How out (caught/bowled/etc.)',
    'fielder': 'Fielder involved'
}

for col in df_deliveries.columns:
    dtype = str(df_deliveries[col].dtype)
    desc = column_descriptions.get(col, 'Column description')
    print(f"{col:<20} | {dtype:<12} | {desc}")

# First look at actual data
print(f"\n👀 FIRST 5 DELIVERIES FROM ACTUAL DATASET:")
print(df_deliveries.head())

# Basic data quality check
print(f"\n🔍 DATA QUALITY OVERVIEW:")
print(f"• Missing values per column:")
missing_data = df_deliveries.isnull().sum()
for col, missing in missing_data.items():
    if missing > 0:
        print(f"  - {col}: {missing:,} missing ({missing/len(df_deliveries)*100:.1f}%)")

🏏 IPL Ball-by-Ball Analytics - Real Kaggle Dataset (260,921 deliveries!)

📊 MODULE 1: LOADING THE REAL IPL DELIVERIES DATASET
--------------------------------------------------
✅ Successfully loaded deliveries.csv!
📈 Dataset size: 260,920 deliveries, 17 columns

🔍 DATASET OVERVIEW:
• Total deliveries: 260,920
• Unique matches: 1,095
• Seasons covered: Likely 2008-2020+ (based on common Kaggle datasets)
• Teams involved: 19 teams

📋 COLUMNS IN OUR DATASET:
Column Name          | Data Type    | Description
-----------------------------------------------------------------
match_id             | int64        | Unique match identifier
inning               | int64        | 1st or 2nd innings
batting_team         | object       | Team currently batting
bowling_team         | object       | Team currently bowling
over                 | int64        | Over number (0-19)
ball                 | int64        | Ball in over (1-6+)
batter               | object       | Batsman on strike
bowler      

In [7]:
# ==========================================
# MODULE 2: DATA MODELLING WITH REAL DATA
# ==========================================

print(f"🏗️ MODULE 2: DATA MODELLING - UNDERSTANDING THE STRUCTURE")
print("-" * 50)

print("🎯 KEY INSIGHTS ABOUT OUR DATA MODEL:")
print("• Each row = ONE BALL bowled in IPL history")
print("• Hierarchical structure: Match → Inning → Over → Ball")
print("• Multiple entities: Players, Teams, Matches, Performances")

# Let's understand the data structure better
print(f"\n📊 DATA STRUCTURE ANALYSIS:")

# Time span analysis
unique_matches = df_deliveries['match_id'].nunique()
total_deliveries = len(df_deliveries)
avg_deliveries_per_match = total_deliveries / unique_matches

print(f"• Unique matches: {unique_matches:,}")
print(f"• Average deliveries per match: {avg_deliveries_per_match:.0f}")
print(f"• This suggests ~{avg_deliveries_per_match/6:.0f} overs per match on average")

# Team analysis
teams = df_deliveries['batting_team'].unique()
print(f"• Teams in dataset: {len(teams)}")
print(f"• Team names: {', '.join(sorted(teams)[:5])}... (showing first 5)")

# Player analysis  
unique_batters = df_deliveries['batter'].nunique()
unique_bowlers = df_deliveries['bowler'].nunique()
print(f"• Unique batters: {unique_batters:,}")
print(f"• Unique bowlers: {unique_bowlers:,}")


🏗️ MODULE 2: DATA MODELLING - UNDERSTANDING THE STRUCTURE
--------------------------------------------------
🎯 KEY INSIGHTS ABOUT OUR DATA MODEL:
• Each row = ONE BALL bowled in IPL history
• Hierarchical structure: Match → Inning → Over → Ball
• Multiple entities: Players, Teams, Matches, Performances

📊 DATA STRUCTURE ANALYSIS:
• Unique matches: 1,095
• Average deliveries per match: 238
• This suggests ~40 overs per match on average
• Teams in dataset: 19
• Team names: Chennai Super Kings, Deccan Chargers, Delhi Capitals, Delhi Daredevils, Gujarat Lions... (showing first 5)
• Unique batters: 673
• Unique bowlers: 530


In [9]:
# ==========================================
# MODULE 3: DESCRIPTIVE STATISTICS ON REAL DATA
# ==========================================

print(f"📈 MODULE 3: DESCRIPTIVE STATISTICS - REAL IPL INSIGHTS")
print("-" * 50)

# Basic scoring statistics
print("🏏 BASIC SCORING STATISTICS:")
runs_stats = df_deliveries['total_runs'].describe()
print(runs_stats)

print(f"\n🎯 WHAT THIS TELLS US:")
print(f"• Average runs per ball: {runs_stats['mean']:.2f}")
print(f"• Most runs in a single delivery: {runs_stats['max']:.0f}")
print(f"• 75% of deliveries score {runs_stats['75%']:.0f} runs or less")

# Boundary analysis
boundaries = df_deliveries[df_deliveries['batsman_runs'].isin([4, 6])]
fours = len(df_deliveries[df_deliveries['batsman_runs'] == 4])
sixes = len(df_deliveries[df_deliveries['batsman_runs'] == 6])

print(f"\n🎯 BOUNDARY ANALYSIS:")
print(f"• Total boundaries: {len(boundaries):,} ({len(boundaries)/len(df_deliveries)*100:.1f}% of all balls)")
print(f"• Fours: {fours:,} ({fours/len(df_deliveries)*100:.1f}%)")
print(f"• Sixes: {sixes:,} ({sixes/len(df_deliveries)*100:.1f}%)")
print(f"• Boundary ratio (4s:6s): {fours/sixes:.1f}:1")

# Wicket analysis
wickets = df_deliveries[df_deliveries['is_wicket'] == 1]
wicket_rate = len(wickets) / len(df_deliveries) * 100

print(f"\n🎯 WICKET ANALYSIS:")
print(f"• Total wickets: {len(wickets):,}")
print(f"• Wicket rate: {wicket_rate:.2f}% (1 wicket every {len(df_deliveries)/len(wickets):.0f} balls)")

# Most common dismissal types
if len(wickets) > 0:
    dismissal_types = wickets['dismissal_kind'].value_counts().head()
    print(f"• Most common dismissals:")
    for dismissal, count in dismissal_types.items():
        print(f"  - {dismissal}: {count:,} ({count/len(wickets)*100:.1f}%)")

# Extras analysis
extras = df_deliveries[df_deliveries['extra_runs'] > 0]
print(f"\n🎯 EXTRAS ANALYSIS:")
print(f"• Deliveries with extras: {len(extras):,} ({len(extras)/len(df_deliveries)*100:.1f}%)")

if len(extras) > 0:
    extras_types = extras['extras_type'].value_counts()
    print(f"• Types of extras:")
    for extra_type, count in extras_types.items():
        if pd.notna(extra_type):  # Skip NaN values
            print(f"  - {extra_type}: {count:,}")


📈 MODULE 3: DESCRIPTIVE STATISTICS - REAL IPL INSIGHTS
--------------------------------------------------
🏏 BASIC SCORING STATISTICS:
count    260920.000000
mean          1.332807
std           1.626416
min           0.000000
25%           0.000000
50%           1.000000
75%           1.000000
max           7.000000
Name: total_runs, dtype: float64

🎯 WHAT THIS TELLS US:
• Average runs per ball: 1.33
• Most runs in a single delivery: 7
• 75% of deliveries score 1 runs or less

🎯 BOUNDARY ANALYSIS:
• Total boundaries: 42,901 (16.4% of all balls)
• Fours: 29,850 (11.4%)
• Sixes: 13,051 (5.0%)
• Boundary ratio (4s:6s): 2.3:1

🎯 WICKET ANALYSIS:
• Total wickets: 12,950
• Wicket rate: 4.96% (1 wicket every 20 balls)
• Most common dismissals:
  - caught: 8,063 (62.3%)
  - bowled: 2,212 (17.1%)
  - run out: 1,114 (8.6%)
  - lbw: 800 (6.2%)
  - caught and bowled: 367 (2.8%)

🎯 EXTRAS ANALYSIS:
• Deliveries with extras: 14,125 (5.4%)
• Types of extras:
  - wides: 8,380
  - legbyes: 4,001
  - no

In [11]:
# ==========================================
# MODULE 4: ADVANCED QUERIES ON REAL DATA
# ==========================================

print(f"\n\n🔍 MODULE 4: ADVANCED QUERIES - EXTRACTING INSIGHTS")
print("-" * 50)

print("🎯 ADVANCED CRICKET ANALYTICS QUERIES:")

# Query 1: Most prolific batsmen
print("\n1️⃣ TOP 10 RUN SCORERS IN IPL HISTORY:")
batsman_runs = df_deliveries.groupby('batter')['batsman_runs'].sum().sort_values(ascending=False)
top_scorers = batsman_runs.head(10)
for i, (batsman, runs) in enumerate(top_scorers.items(), 1):
    print(f"   {i:2d}. {batsman:<25} : {runs:,} runs")

# Query 2: Most economical bowlers (minimum 500 balls bowled)
print("\n2️⃣ MOST ECONOMICAL BOWLERS (Min 500 balls):")
bowler_stats = df_deliveries.groupby('bowler').agg({
    'total_runs': 'sum',
    'ball': 'count'
}).reset_index()

bowler_stats = bowler_stats[bowler_stats['ball'] >= 500]  # Minimum 500 balls
bowler_stats['economy'] = (bowler_stats['total_runs'] / bowler_stats['ball']) * 6
top_economical = bowler_stats.nsmallest(10, 'economy')

for i, row in enumerate(top_economical.itertuples(), 1):
    print(f"   {i:2d}. {row.bowler:<25} : {row.economy:.2f} economy ({row.ball} balls)")

# Query 3: Highest scoring overs
print("\n3️⃣ HIGHEST SCORING OVERS:")
over_runs = df_deliveries.groupby(['match_id', 'inning', 'over'])['total_runs'].sum().sort_values(ascending=False)
highest_overs = over_runs.head(10)
print("   Match ID  | Inning | Over | Runs")
print("   " + "-"*35)
for (match_id, inning, over), runs in highest_overs.items():
    print(f"   {match_id:<9} | {inning:6} | {over:4} | {runs:4}")

# Query 4: Team performance analysis
print("\n4️⃣ TEAM BATTING AVERAGES:")
team_batting = df_deliveries.groupby('batting_team')['batsman_runs'].agg(['sum', 'count', 'mean']).round(2)
team_batting = team_batting.sort_values('sum', ascending=False)
team_batting.columns = ['Total_Runs', 'Balls_Faced', 'Avg_per_Ball']

print("   Team                        | Total Runs | Balls  | Avg/Ball")
print("   " + "-"*65)
for team, row in team_batting.head(10).iterrows():
    print(f"   {team:<27} | {row['Total_Runs']:>10,} | {row['Balls_Faced']:>6,} | {row['Avg_per_Ball']:>8.2f}")

# Query 5: PowerPlay vs Death Over Analysis
print("\n5️⃣ POWERPLAY vs DEATH OVERS COMPARISON:")
powerplay = df_deliveries[df_deliveries['over'] <= 5]  # First 6 overs
death_overs = df_deliveries[df_deliveries['over'] >= 16]  # Last 4 overs

pp_avg = powerplay['total_runs'].mean()
death_avg = death_overs['total_runs'].mean()

print(f"   PowerPlay (Overs 1-6):")
print(f"   • Average runs per ball: {pp_avg:.2f}")
print(f"   • Total deliveries: {len(powerplay):,}")
print(f"   • Wickets: {len(powerplay[powerplay['is_wicket']==1]):,}")

print(f"   Death Overs (Overs 17-20):")
print(f"   • Average runs per ball: {death_avg:.2f}")
print(f"   • Total deliveries: {len(death_overs):,}")
print(f"   • Wickets: {len(death_overs[death_overs['is_wicket']==1]):,}")




🔍 MODULE 4: ADVANCED QUERIES - EXTRACTING INSIGHTS
--------------------------------------------------
🎯 ADVANCED CRICKET ANALYTICS QUERIES:

1️⃣ TOP 10 RUN SCORERS IN IPL HISTORY:
    1. V Kohli                   : 8,014 runs
    2. S Dhawan                  : 6,769 runs
    3. RG Sharma                 : 6,630 runs
    4. DA Warner                 : 6,567 runs
    5. SK Raina                  : 5,536 runs
    6. MS Dhoni                  : 5,243 runs
    7. AB de Villiers            : 5,181 runs
    8. CH Gayle                  : 4,997 runs
    9. RV Uthappa                : 4,954 runs
   10. KD Karthik                : 4,843 runs

2️⃣ MOST ECONOMICAL BOWLERS (Min 500 balls):
    1. A Kumble                  : 6.65 economy (983 balls)
    2. M Muralitharan            : 6.70 economy (1581 balls)
    3. SP Narine                 : 6.76 economy (4146 balls)
    4. DW Steyn                  : 6.79 economy (2282 balls)
    5. DL Vettori                : 6.83 economy (785 balls)
    6. Ra

In [63]:
# ==========================================
# PRACTICAL EXERCISES
# ==========================================

print(f"\n\n🎯 PRACTICAL EXERCISES FOR YOU TO TRY:")
print("-" * 50)
print("1. Find the bowler who has taken the most wickets")
most_wickets = df_deliveries[df_deliveries['is_wicket']==1].groupby('bowler').size().sort_values(ascending=False).head(1)
top_bowler_name = most_wickets.index[0]
print(top_bowler_name)
print("2. Which team has the best PowerPlay record?") 
#best_pp = df_deliveries[df_deliveries['over']<=5].groupby('batting').size()
best_pp = df_deliveries[df_deliveries['over'] <= 5].groupby('batting_team')['total_runs'].sum().sort_values(ascending=False).index[0]
print(best_pp)
print("3. Who has hit the most sixes in IPL history?")
most_sixes = df_deliveries[df_deliveries['batsman_runs']==6].groupby('batter').size().sort_values(ascending=False).head(1).index[0]
print(most_sixes)
print("4. What's the average runs scored in the 20th over?")
avg_20over = df_deliveries[df_deliveries['over']==19]['total_runs'].mean()
print(avg_20over)
print("5. Which batsman has the best strike rate (min 1000 balls)?")
# Step 1: Group by batter
batsman_stats = df_deliveries.groupby('batter').agg(runs=('batsman_runs', 'sum'),balls=('batsman_runs', 'count'))  # counting all deliveries faced)
# Step 2: Filter for batters with at least 1000 balls
qualified_batsmen = batsman_stats[batsman_stats['balls'] >= 1000].copy()
# Step 3: Calculate strike rate
qualified_batsmen['strike_rate'] = (qualified_batsmen['runs'] / qualified_batsmen['balls']) * 100
# Step 4: Find the batsman with the best strike rate
top_striker = qualified_batsmen.sort_values('strike_rate', ascending=False).head(1)
# Step 5: Print result
print(top_striker[['strike_rate']])

# 1. Find matches where teams scored 200+ runs
high_scoring = df_deliveries.groupby(['match_id', 'inning', 'batting_team'])['total_runs'].sum()
high_scoring[high_scoring >= 200]
print('\nHigh scoring matches')
print(high_scoring)

# 2. Bowlers with best economy in death overs
death_bowling = df_deliveries[df_deliveries['over'] >= 16]
death_economy = death_bowling.groupby('bowler').agg({
    'total_runs': 'sum', 'ball': 'count'
})
print('\nBest death bowling')
print(death_bowling)
print('\nBest death economy')
print(death_economy)

# 3. Most consistent batsmen (lowest coefficient of variation)
batsman_consistency = df_deliveries.groupby('batter')['batsman_runs'].agg(['mean', 'std']).head(10)
print('\nMost consistent Batsmen')
print(batsman_consistency)



🎯 PRACTICAL EXERCISES FOR YOU TO TRY:
--------------------------------------------------
1. Find the bowler who has taken the most wickets
YS Chahal
2. Which team has the best PowerPlay record?
Mumbai Indians
3. Who has hit the most sixes in IPL history?
CH Gayle
4. What's the average runs scored in the 20th over?
1.7768553710742148
5. Which batsman has the best strike rate (min 1000 balls)?
            strike_rate
batter                 
AD Russell   164.224422

High scoring matches
match_id  inning  batting_team               
335982    1       Kolkata Knight Riders          222
          2       Royal Challengers Bangalore     82
335983    1       Chennai Super Kings            240
          2       Kings XI Punjab                207
335984    1       Rajasthan Royals               129
                                                ... 
1426310   2       Rajasthan Royals               174
1426311   1       Sunrisers Hyderabad            175
          2       Rajasthan Royals     